# ------------------------------ TANDEM model ------------------------------
Make sure the machine has more than 24 GB RAM and 8 CPU cores for running this code

This code uses knowledge graph representations of patients (called patient SPOKEsig) which you have to download before running this notebook (as mentioned in the [README instructions](https://github.com/BaranziniLab/TANDEM#instructions)).

In [6]:
from utility import *

# Load model 
### ------------------------------ Note --------------------------------
       train_flag = False --> Use pre-trained model
       train_flag = True --> Train a new model 

In [11]:
temporal_model = load_model_type(model_type="temporal", train_flag=False)
non_temporal_model = load_model_type(model_type="non_temporal", train_flag=False)
tandem_model = load_tandem(temporal_model, non_temporal_model, train_flag=False)


Selected to train temporal model
Loading training data for temporal model...
Training data is loaded!


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   54.4s


KeyboardInterrupt: 

# Compute model predictions

In [8]:
y_score_temporal = get_predictions(temporal_model, temporal_test_data)
print("AUC score of temporal predictions = ", round(get_auc(test_metadata.label.values, y_score_temporal), 2))

y_score_non_temporal = get_predictions(non_temporal_model, non_temporal_test_data)
print("AUC score of non-temporal predictions = ", round(get_auc(test_metadata.label.values, y_score_non_temporal), 2))

y_score_tandem, optThresh = get_tandem_predictions(tandem_model, 
                                                   temporal_model, 
                                                   non_temporal_model, 
                                                   temporal_test_data, 
                                                   non_temporal_test_data, 
                                                   test_metadata)
print("AUC score of TANDEM predictions = ", round(get_auc(test_metadata.label.values, y_score_tandem), 2))

AUC score of temporal predictions =  0.8
AUC score of non-temporal predictions =  0.73
63/63 [==============================] - 0s 905us/step
AUC score of TANDEM predictions =  0.85


In [9]:
# PD patient example
patient_index = 14
get_tandem_single_patient_prediction(patient_index, 
                                     optThresh, 
                                     temporal_model, 
                                     non_temporal_model, 
                                     tandem_model, 
                                     test_metadata)

1/1 [==============================] - 0s 19ms/step
********** Patient prediction ***********
Label :  PD
TANDEM prediction :  PD
TANDEM prediction score : 0.75


In [10]:
# Non-PD patient example
patient_index = 34
get_tandem_single_patient_prediction(patient_index, 
                                     optThresh, 
                                     temporal_model, 
                                     non_temporal_model, 
                                     tandem_model, 
                                     test_metadata)

1/1 [==============================] - 0s 19ms/step
********** Patient prediction ***********
Label :  Non-PD
TANDEM prediction :  Non-PD
TANDEM prediction score : 0.04
